
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>



# Building Multi-stage AI Systems in Databricks

In this demo we will start building a multi-stage reasoning system using Databricks' features and LangChain. Before we build the chain, first, we will show various components that are commonly used in multi-stage chaining system. 

In the main section of the demo, we will build a multi-stage system. First, we will build a chain that will answer user questions using DBRX model. The second chain will search for DAIS-2023 talks and will try to find the corresponding video on YouTube. The final, complete chain will recommend videos to the user.

**Learning Objectives:**

*By the end of this demo, you will be able to;*

* Identify that LangChain can include stages/tasks that are not LLMs.

* Create basic LLM chains to connect prompts and LLMs.

* Use tools to complete various tasks in the complete system.

* Construct sequential chains of multiple LLMChains to perform multi-stage reasoning analysis.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **14.3.x-cpu-ml-scala2.12 14.3.x-scala2.12**



## Classroom Setup

Before starting the demo, run the provided classroom setup script. This script will define configuration variables necessary for the demo. Execute the following cell:

In [0]:
%pip install -U --quiet databricks-sdk==0.29.0 langchain-core==0.2.24 databricks-vectorsearch==0.40 langchain-community==0.2.10 typing-extensions==4.12.2 youtube_search Wikipedia grandalf mlflow==2.14.3 pydantic==2.8.2

dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-feature-engineering 0.2.1 requires pyspark<4,>=3.1.2, which is not installed.
ydata-profiling 4.2.0 requires pydantic<2,>=1.8.1, but you have pydantic 2.8.2 which is incompatible.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-02

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| dropping the catalog "labuser7793232_1dy8_da"...(1 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/generative-ai-application-development/v01"

Validating the locally installed datasets:
| listing local files...(0 seconds)
| validation completed...(0 seconds total)
Creating & using the catalog "labuser7793232_1dy8_da"...(2 seconds)
Dataset is created successfully.

Predefined tables in "labuser7793232_1dy8_da.default":
| dais_text

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/labuser7793232@vocareum.com/generative-ai-application-development
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/generative-ai-application-development/v01

Setup completed (37 seconds)

The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmfu

**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser7793232@vocareum.com
Catalog Name:      labuser7793232_1dy8_da
Schema Name:       default
Working Directory: dbfs:/mnt/dbacademy-users/labuser7793232@vocareum.com/generative-ai-application-development
Dataset Location:  dbfs:/mnt/dbacademy-datasets/generative-ai-application-development/v01


## Using LLMs and Prompts without an External Library

While there are many libraries out there for building chains and in this demo we will be using one as well, you don't need a third party library for a simple prompt. We can simply use `databricks-sdk` to directly query an **Foundational Model API endpoint**. 


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage

w = WorkspaceClient()

genre = "romance"
actor = "Brad Pitt"

prompt = f"Tell me about a {genre} movie which {actor} is one of the actors."

messages = [
    { 
        "role": "user", 
        "content": prompt 
    }
]
messages = [ChatMessage.from_dict(message) for message in messages]
llm_response = w.serving_endpoints.query(
    name="databricks-dbrx-instruct",
    messages=messages,
    temperature=0.2,
    max_tokens=128
)

print(llm_response.as_dict()["choices"][0]["message"]["content"])

One popular romance movie that Brad Pitt starred in is "The Curious Case of Benjamin Button" (2008). The film is a romantic fantasy drama directed by David Fincher, based on a short story by F. Scott Fitzgerald. Brad Pitt plays the role of Benjamin Button, a man who is born with the appearance and physical limitations of an elderly person and ages in reverse. The movie follows his life as he falls in love with Daisy, played by Cate Blanchett, and the challenges they face due to Benjamin's unusual aging process. It's a beautiful and poignant story about love, life, and the passage of time.



## LangChain Basics

As demonstrated in the previous section, **it is not necessary to use a third-party chaining library** to construct a multi-chain AI system. However, composition libraries like **LangChain** can simplify some of the steps by providing a generic interface for supported large language models (LLMs).

Before we begin building a multi-stage chain, let's review the main LangChain components that we will use in this module.


### Prompt

Prompt is one of the basic blocks when interacting with GenAI models. They may include instructions, examples and specific context information related to the given task. Let's create a very basic prompt.


In [0]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me about a {genre} movie which {actor} is one of the actors.")
prompt_template.format(genre="romance", actor="Brad Pitt")

'Tell me about a romance movie which Brad Pitt is one of the actors.'

### LLMs

LLMs are the core component when building compound AI systems. They are the **brain** of the system for reasoning and generating the response. The [list of supported LLMs](https://python.langchain.com/v0.2/docs/integrations/chat/) can be found on LangChain documentation. 

Let's see how to interact with **Databricks's DBRX** model.


In [0]:
from langchain_community.chat_models import ChatDatabricks

# You can play with max_tokens to define the length of the response
llm_dbrx = ChatDatabricks(endpoint="databricks-dbrx-instruct", max_tokens = 500)

for chunk in llm_dbrx.stream("Who is Brad Pitt?"):
    print(chunk.content, end="\n", flush=True)

Brad
 Pitt
 is
 an
 American
 actor
 and
 film
 producer
.
 He
 has
 received
 multiple
 awards
 and
 nominations
,
 including
 two
 Academy
 Awards
 and
 a
 Golden
 Globe
 Award
.
 He
 is
 known
 for
 his
 roles
 in
 various
 films
,
 such
 as
 "
Fight
 Club
,"
 "
Ocean
's
 Eleven
,"
 and
 "
Once
 Upon
 a
 Time
 in
 Hollywood
."



### Retriever

Retrievers accept a string as input and return the list of documents. They are mainly used for retrieving external data and passing it to the model for generating response. Typically, retrievers don't use LLMs instead they are using similarity search under the hood to find the most similar documents. There are various types of retrievers such as *document retrievers* and *vector store retrievers*. The [list of supported retrievers](https://python.langchain.com/v0.2/docs/integrations/retrievers/) can be found on LangChain documentation. 

In the next section of the demo, we will use **Databricks Vector Search** as retriever to fetch documents by input query.

For now, let's try a simple **Wikipedia retriever**.

In [0]:
from langchain_community.retrievers import WikipediaRetriever
retriever = WikipediaRetriever()
docs = retriever.invoke(input="Brad Pitt")
print(docs[0])

page_content='William Bradley Pitt (born December 18, 1963) is an American actor and film producer. He is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, two Golden Globe Awards, and a Primetime Emmy Award. One of the most influential celebrities, Pitt appeared on Forbes' annual Celebrity 100 list  from 2006 to 2008, and the Time 100 list in 2007.
Pitt first gained recognition as a cowboy hitchhiker in the Ridley Scott road film Thelma & Louise (1991). Pitt emerged as a star taking on leading man roles in films such as the drama A River Runs Through It (1992), the western Legends of the Fall (1994), the horror film Interview with the Vampire (1994), and the crime thriller Seven (1995). Pitt found greater commercial success starring in Steven Soderbergh's heist film Ocean's Eleven (2001), and reprised his role in its sequels. He cemented his leading man status starring in blockbusters such as the historical epic Troy (2004), the romanti

/local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


### Tools

Tools are functions that can be invoked in the chain. Tools has *input parameters* and a *function* to run. The [list of supported tools](https://python.langchain.com/v0.2/docs/integrations/tools/) can be found on LangChain documentation. 

Here, we have a Youtube search tool. The tool's `description` defines why a tool can be used and the `args` defines what input arguments can be passed to the tool.

In [0]:
from langchain_community.tools import YouTubeSearchTool
tool = YouTubeSearchTool()
tool.run("Brad Pitt movie trailer")

"['https://www.youtube.com/watch?v=wLJUPjiRbAM&pp=ygUXQnJhZCBQaXR0IG1vdmllIHRyYWlsZXI%3D', 'https://www.youtube.com/watch?v=DrE9umGiPZQ&pp=ygUXQnJhZCBQaXR0IG1vdmllIHRyYWlsZXI%3D']"

In [0]:
print(tool.description)
print(tool.args)

search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional
{'query': {'title': 'Query', 'type': 'string'}}


### Chaining

One of the important features of these components is the ability to **chain** them together. Let's connect the LLM with the prompt.

In [0]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm_dbrx | StrOutputParser()
print(chain.invoke({"genre":"romance", "actor":"Brad Pitt"}))

One popular romance movie that Brad Pitt starred in is "The Curious Case of Benjamin Button" (2008). The film is a romantic fantasy drama directed by David Fincher, based on a short story by F. Scott Fitzgerald. Brad Pitt plays the role of Benjamin Button, a man who is born with the appearance and physical limitations of an elderly person and ages in reverse. The movie follows his life as he falls in love with Daisy, played by Cate Blanchett, and the challenges they face due to Benjamin's unusual aging process. It's a beautiful and poignant story about love, life, and the passage of time.


## Build a Multi-stage Chain

### Enable MLflow Auto-Log

MLflow has support for auto-logging LangChain models. Before we start building the chains, we will enable auto-logging below.

In [0]:
import mlflow
mlflow.langchain.autolog()

### Create a Vector Store

This course does not cover vector stores in depth, as they fall outside its scope. For a comprehensive understanding of vector stores, including the Databricks Vector Store, please refer to the **Generative AI Solution Development** course. 

**🚨IMPORTANT: Vector Search endpoints must be created before running the rest of the demo. Endpoint names should be in this format; `vs_endpoint_x`. The endpoint will be assigned by username.**

In [0]:
# Assign VS search endpoint by username
vs_endpoint_prefix = "vs_endpoint_"
vs_endpoint_fallback = "vs_endpoint_fallback"
vs_endpoint_name = vs_endpoint_prefix+str(get_fixed_integer(DA.unique_name("_")))
print(f"Vector Endpoint name: {vs_endpoint_name}. In case of any issues, replace variable `vs_endpoint_name` with `vs_endpoint_fallback` in demos and labs.")

# Source table and VS index table names
vs_index_table_fullname = f"{DA.catalog_name}.{DA.schema_name}.dais_embeddings"
source_table_fullname = f"{DA.catalog_name}.{DA.schema_name}.dais_text"

Vector Endpoint name: vs_endpoint_9. In case of any issues, replace variable `vs_endpoint_name` with `vs_endpoint_fallback` in demos and labs.


The dataset that we will be using in this module is already created in the classroom setup script. Let's have a quick look at the dataset.

Then, we will create a vector store index and store embeddings there.

In [0]:
display(spark.sql(f"SELECT * FROM {source_table_fullname}"))

Title,Abstract,id
Nebula: The Journey of Scaling Instacart’s Data Pipelines with Apache Spark™ and Lakehouse,"Instacart has gone through immense growth during the pandemic and the trend continues. Instacart ads is no exception in this growth story. We have launched many new product lines including display and video ads covering the full advertising funnel to address the increasing demand of our retail partners. We have built advanced models to auto-suggest optimal bidding to increase the ROI for our CPG partners. Advertisers’ trust is the utmost priority and thus the quest to build a top-class ads measurement platform. Ads data processing requires complex data verifications to update ads serving stats. In ETL pipelines these were implemented through files containing thousands of lines of raw SQL which were hard to scale, test, and iterate upon. Our data engineers used to spend hours testing small changes due to a lack of local testing mechanisms. These pain points stress our need for better tools. After some research, we chose Apache Spark™ as our preferred tool to rebuild ETLs, and the Databricks platform made this move easier. In this presentation, I will share our journey to move our pipelines to Spark and Delta Lake on Databricks. With spark, scala, and delta we solved many problems which were slowing the team’s productivity. Some key areas I will cover include: - Modular and composable code - Unit testing framework - Incremental event processing with spark structured streaming - Granular resource tuning for better performance and cost efficacy Other than the domain business logic, the problems discussed here are quite common for performing data processing at scale. I would be glad to have the opportunity to share our learning and am hopeful that it will benefit others who are going through similar growth challenges or migrating to Lakehouse.",0
Satellite Imaginary Data Processing Using Apache Spark™ and H3 Geospatial Indexing System,"Agriculture is a complex ecosystem. Understanding Ag data around soil metrics, weather and historical crop production is a key to adopt sustainable practices which help farmers to enhance their profitability and soil health. As these datasets are huge and disparate in nature, finding a standard unit of analysis and bringing all the data to a common granularity is challenging. By leveraging the distributed data processing capabilities of Apache Spark™ and h3 geospatial indexing system, created a hexagonal grid and mapped all the data sets using h3 index. This gave us an ability to join all the datasets together and helped us in deriving more insights. This session will share our learnings and experiences with the Spark community.",1
From Snowflake to Enterprise-Scale Apache Spark™,"Akamai mPulse is a real user monitoring (RUM) solution that delivers real-time web performance analytics to Akamai customers through dashboards, alerting, reporting and data science. The architecture of mPulse relies on a combination of public and private cloud-based services, such as Amazon AWS, Microsoft Azure and the Snowflake data warehouse. Snowflake has provided the core data warehousing needs as the product has grown at scale along with Akamai’s customers. The engineering team at mPulse has been re-architecting the system to migrate away from Snowflake to an internal enterprise-scale Apache Spark™ solution that Akamai has been developing in-house to improve performance and save on cost. In the first half of the talk, we’ll discuss how the mPulse team made the decision to migrate, the challenges we’ve seen and how Spark is suiting the product's needs. In the second half of the talk, we’ll discuss the details of the Spark-based infrastructure. Akamai data warehouse (a.k.a Asgard) is a Spark-based solution running on the Azure cloud. We will describe the internal and unique technologies and characteristics of the solution that enable it to outperform Snowflake's offering both from a cost and performance perspect

In [0]:
# Store embeddings in vector store
create_vs_index(vs_endpoint_name, vs_index_table_fullname, source_table_fullname, "Title")

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Endpoint named vs_endpoint_9 is ready.
Creating index labuser7793232_1dy8_da.default.dais_embeddings on endpoint vs_endpoint_9...
Waiting for index to be ready, this can take a few min... {'detailed_state': 'PROVISIONING_INDEX', 'message': 'Delta sync Index creation is pending. Check latest status: https://dbc-5207d6f8-be41.cloud.databricks.com/explore/data/labuser7793232_1dy8_da/default/dais_embeddings', 'indexed_row_count': 0, 'ready': False, 'index_url': 'dbc-5207d6f8-be41.cloud.databricks.com/api/2.0/vector-search/endpoints/vs_endpoint_9/indexes/labuser7793232_1dy8_da.default.dais_embeddings'} - pipeline url:dbc-5207d6f8-be41.cloud.databricks.com/api/2.0/vector-search/endpoints/vs_endpoint_9/indexes/labuser7793232_1dy8_da.default.dais_embeddings


### Define Common Objects

In [0]:
from langchain_community.chat_models import ChatDatabricks
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, HTML

llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-70b-instruct", max_tokens = 1000)

### Build First Chain

The **first chain** will be used for listing videos relevant to the user's question. In order to get videos, first, we need to search for the DAIS-2023 talks that are already stored in a Vector Search index. After retrieving the relevant titles, we will use YouTube search tool to get the videos for the talks. In the final stage, these videos are passed to the chain to generate a response for the user.

This chain consist of a `prompt template`, `retriever`, `llm model` and `output parser`.

In [0]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import DatabricksVectorSearch

# Prompt to format the video titles for YouTube search
prompt_template_1 = PromptTemplate.from_template(
    """
    Construct a search query for YouTube based on the titles below. Make sure to add DAIS 2023 as part of the query. Remove all quotes from the query and return only the query.

    <video_titles>
    {context}
    </video_titles>

    Answer:
    """
)

# Create a vector store client and retrieve documents
def get_retriever(persist_dir=None):
    vsc = VectorSearchClient(disable_notice=True)
    vs_index = vsc.get_index(vs_endpoint_name, vs_index_table_fullname)
    vectorstore = DatabricksVectorSearch(vs_index, text_column="Title")
    return vectorstore.as_retriever(search_kwargs={"k": 2})


# First chain
chain_video = (
    {"context": get_retriever(), "input": RunnablePassthrough()}
    | prompt_template_1
    | llm_llama
    | StrOutputParser()
)

# Test the chain
chain_video.invoke("How machine learning models are stored in Unity Catalog?")

'Unity Catalog at Scale in Retail Data Engineering and Data Science DAIS 2023 OR Post-Merger: Implementing Unity Catalog Across Multiple Accounts DAIS 2023'

Trace(info=TraceInfo(request_id='tr-3ed1b953767442d1abd58def8b7c08f1', experiment_id='295899001271457', timestamp_ms=1727761225066, execution_time_ms=1283, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.trace_schema.version': '2', 'mlflow.traceInputs': '"How machine learning models are stored in Unity Catalog?"', 'mlflow.traceOutputs': '"Unity Catalog at Scale in Retail Data Engineering and Data Science DAIS 2023 OR Post-Merger: Implementing Unity Catalog Across Multiple Accounts DAIS 2023"'}, tags={'mlflow.databricks.notebook.commandID': '7070825677414328105_5713943020515797726_435d7bddbae54a27a8e8814c76d7296f', 'mlflow.databricks.notebookID': '295899001271457', 'mlflow.databricks.notebookPath': '/Users/labuser7793232@vocareum.com/generative-ai-application-development-1.1.1/02 - Building Multi-stage Reasoning/2.1 - Building Multi-stage AI Systems', 'mlflow.databricks.webappURL': 'https://oregon.cloud.databricks.com', 'mlflow.databricks.workspaceID': '3345972304455344', 'mlfl

### Build Second Chain

This chain will use the video title-based query from the previous chain to search YouTube. The response from this chain will consist of YouTube video links.

In [0]:
from langchain_community.tools import YouTubeSearchTool
from langchain_core.runnables import RunnableLambda

# Generate image using first chain
def get_videos(input):
    tool_yt = YouTubeSearchTool()
    video_urls = tool_yt.run(input)
    return video_urls

chain_youtube = RunnableLambda(get_videos) | StrOutputParser()

# Get the image URL
response = chain_youtube.invoke("DAIS 2023 Streamlining API Deployment for ML Models Across Multiple Brands Ahold Delhaize's Experience on Serverless OR Building a Real-Time Model Monitoring Pipeline on Databricks")

print(response)

['https://www.youtube.com/watch?v=GSJFyoBiCXk&pp=ygW0AURBSVMgMjAyMyBTdHJlYW1saW5pbmcgQVBJIERlcGxveW1lbnQgZm9yIE1MIE1vZGVscyBBY3Jvc3MgTXVsdGlwbGUgQnJhbmRzIEFob2xkIERlbGhhaXplJ3MgRXhwZXJpZW5jZSBvbiBTZXJ2ZXJsZXNzIE9SIEJ1aWxkaW5nIGEgUmVhbC1UaW1lIE1vZGVsIE1vbml0b3JpbmcgUGlwZWxpbmUgb24gRGF0YWJyaWNrcw%3D%3D', 'https://www.youtube.com/watch?v=EhRHQPCdldI&pp=ygW0AURBSVMgMjAyMyBTdHJlYW1saW5pbmcgQVBJIERlcGxveW1lbnQgZm9yIE1MIE1vZGVscyBBY3Jvc3MgTXVsdGlwbGUgQnJhbmRzIEFob2xkIERlbGhhaXplJ3MgRXhwZXJpZW5jZSBvbiBTZXJ2ZXJsZXNzIE9SIEJ1aWxkaW5nIGEgUmVhbC1UaW1lIE1vZGVsIE1vbml0b3JpbmcgUGlwZWxpbmUgb24gRGF0YWJyaWNrcw%3D%3D']


Trace(info=TraceInfo(request_id='tr-adfbcf128a864583af1ef8b8f6f095e0', experiment_id='295899001271457', timestamp_ms=1727761386118, execution_time_ms=1005, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.trace_schema.version': '2', 'mlflow.traceInputs': '"DAIS 2023 Streamlining API Deployment for ML Models Across Multiple Brands Ahold Delhaize\'s Experience on Serverless OR Building a Real-Time Model Monitoring Pipeline on Databricks"', 'mlflow.traceOutputs': '"[\'https://www.youtube.com/watch?v=GSJFyoBiCXk&pp=ygW0AURBSVMgMjAyMyBTdHJlYW1saW5pbmcgQVBJIERlcGxveW1lbnQgZm9yIE1MIE1vZGVscyBBY3Jvc3MgTXVsdGlwbGUgQnJhbmRzIEFob2xkIERlbGhhaXplJ3MgRXhwZXJpZW5jZSBvbiBTZXJ2ZXJsZXNzIE9SIEJ1aWxkaW5nIGEgUmVhbC1UaW1lIE1vZ...'}, tags={'mlflow.databricks.notebook.commandID': '7070825677414328105_5112204143799855281_f0fba4681eef48a6a245260faeae04fe', 'mlflow.databricks.notebookID': '295899001271457', 'mlflow.databricks.notebookPath': '/Users/labuser7793232@vocareum.com/generative-ai-application-de

### Build Third Chain

The **third chain** will be a simple question-answer prompt using **DBRX**. The chain will use the video links as well for recommendation. This chain consist of a `prompt template`, `llm model` and `output parser`.

In [0]:
prompt_template_3 = PromptTemplate.from_template(
    """You are a Databricks expert. You will get questions about Databricks. Try to give simple answers and be professional. Don't include code in your response.

    Question: {input}

    Answer:

    Also, encourage the user to watch the videos provided below. Show video links as a list. Strip the YouTube link at "&pp=" and keep the first part of the URL. There is no title for the links so only show the URL. Only use the videos provided below.

    Video Links: {videos}

    Format response in HTML format.
    """
)

chain_expert = (prompt_template_3 | llm_llama | StrOutputParser())
chain_expert.invoke({
    "input": "How machine learning models are stored in Unity Catalog?",
    "videos": ""
    })

'<p>Machine learning models in Databricks are stored in Unity Catalog as MLflow models. Unity Catalog provides a centralized repository for managing and governing MLflow models, making it easier to track, version, and deploy models across the organization.</p>\n\n<p>For more information on Unity Catalog and MLflow, I recommend watching the following videos:</p>\n\n<ul>\n  <li>https://www.youtube.com/watch?v=dQw4w9WgXcQ</li>\n  <li>https://www.youtube.com/watch?v=jNQXAC9IVRw</li>\n</ul>\n\n<p>These videos provide a detailed overview of Unity Catalog and MLflow, and how they can be used to manage and deploy machine learning models in Databricks.</p>'

Trace(info=TraceInfo(request_id='tr-e3a146deb8624156b604e9637ff5fc9b', experiment_id='295899001271457', timestamp_ms=1727761478719, execution_time_ms=3214, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.trace_schema.version': '2', 'mlflow.traceInputs': '{"input": "How machine learning models are stored in Unity Catalog?", "videos": ""}', 'mlflow.traceOutputs': '"<p>Machine learning models in Databricks are stored in Unity Catalog as MLflow models. Unity Catalog provides a centralized repository for managing and governing MLflow models, making it easier to track, version, and deploy models across the orga...'}, tags={'mlflow.databricks.notebook.commandID': '7070825677414328105_4813913930471059810_aa52908f7fc7409b87eb998e34df1399', 'mlflow.databricks.notebookID': '295899001271457', 'mlflow.databricks.notebookPath': '/Users/labuser7793232@vocareum.com/generative-ai-application-development-1.1.1/02 - Building Multi-stage Reasoning/2.1 - Building Multi-stage AI Systems', 'mlflow.d

### Chaining Chains ⛓️

So far we create chains for each stage. To build a multi-stage system, we need to link these chains together and build a multi-chain system.

In [0]:
multi_chain = (
  {
    "input": RunnablePassthrough(),
    "videos": (chain_video | chain_youtube | StrOutputParser())
  }
  |chain_expert
  |llm_llama 
  |StrOutputParser()
)

query = "How machine learning models are stored in Unity Catalog?"
response = multi_chain.invoke(query)
display(HTML(response))

Trace(info=TraceInfo(request_id='tr-b06067a6aafc4ab3a4f3f006f97c88f1', experiment_id='295899001271457', timestamp_ms=1727761510669, execution_time_ms=11018, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.trace_schema.version': '2', 'mlflow.traceInputs': '"How machine learning models are stored in Unity Catalog?"', 'mlflow.traceOutputs': '"Here is a rewritten version of the text in a more formal and polished tone:\\n\\n**Managing Machine Learning Models with Unity Catalog**\\n\\nUnity Catalog provides a centralized repository for storing and serving machine learning models as managed t...'}, tags={'mlflow.databricks.notebook.commandID': '7070825677414328105_6495440519279234730_970fd14983874ec79c82bc8f660ec817', 'mlflow.databricks.notebookID': '295899001271457', 'mlflow.databricks.notebookPath': '/Users/labuser7793232@vocareum.com/generative-ai-application-development-1.1.1/02 - Building Multi-stage Reasoning/2.1 - Building Multi-stage AI Systems', 'mlflow.databricks.webappURL'

View the flow of the final chain.

In [0]:
multi_chain.get_graph().print_ascii()

                                 +-----------------------------+                          
                                 | Parallel<input,videos>Input |                          
                                 +-----------------------------+                          
                                     ****                      *****                      
                                  ***                               *****                 
                                **                                       ******           
             +------------------------------+                                  ***        
             | Parallel<context,input>Input |                                    *        
             +------------------------------+                                    *        
                    **               **                                          *        
                 ***                   ***                                       *        

## Save the Chain to Model Registery in UC

Now that our chain is ready and evaluated, we can register it within our Unity Catalog schema. 

After registering the chain, you can view the chain and models in the **Catalog Explorer**.

In [0]:
from mlflow.models import infer_signature
import mlflow


# Set model registery to UC
mlflow.set_registry_uri("databricks-uc")
model_name = f"{DA.catalog_name}.{DA.schema_name}.multi_stage_demo"

with mlflow.start_run(run_name="multi_stage_demo") as run:
    signature = infer_signature(query, response)
    model_info = mlflow.langchain.log_model(
        multi_chain,
        loader_fn=get_retriever, 
        artifact_path="chain",
        registered_model_name=model_name,
        input_example=query,
        signature=signature
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/mlflow/langchain/__init__.py:581: UserWarning: If you are constructing a custom LangChain model, please upgrade cloudpickle to version 2.1.0 or later using `pip install cloudpickle>=2.1.0` to ensure the model can be loaded correctly.
  warnings.warn(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.3. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/langchain_community/llms/__init__.py:173: 

Uploading artifacts:   0%|          | 0/28 [00:00<?, ?it/s]

Successfully registered model 'labuser7793232_1dy8_da.default.multi_stage_demo'.


Uploading artifacts:   0%|          | 0/28 [00:00<?, ?it/s]

Created version '1' of model 'labuser7793232_1dy8_da.default.multi_stage_demo'.


## Load the chain from Model Registery in UC

Now that our chain is registered in UC, we can load it and invoke it.

In [0]:
model_uri = f"models:/{model_name}/{model_info.registered_model_version}"
model = mlflow.langchain.load_model(model_uri)

model.invoke("How machine learning models are stored in Unity Catalog?")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.3. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/local_disk0/.ephemeral_nfs/envs/pythonEnv-e96a1264-737c-4930-9d3d-e91c58ab473d/lib/python3.10/site-packages/langchain_community/llms/__init__.py:173: LangChainDeprecationWarning: `` was deprecated in LangChain 0.0.22 and will be removed in 0.3. An updated version of the  exists in the langchain-community package and should be used instead. To use it run `pip install -U langchain-community` and import as `from langchain_community.chat_models import ChatDatabricks`.
  warn_deprecated(
/local_disk0/.ephemera

'The provided text is a passage that describes how machine learning models are stored and managed in Unity Catalog. Here\'s a breakdown of the content:\n\n**Main Points:**\n\n1. Machine learning models are stored in Unity Catalog as managed tables, optimized for storing and serving models.\n2. Unity Catalog provides a centralized repository for managing and versioning machine learning models.\n3. The catalog makes it easier to track changes and collaborate with others.\n\n**Additional Resources:**\n\nThe passage recommends watching two YouTube videos for more information on Unity Catalog and machine learning model management. The videos are:\n\n1. "https://www.youtube.com/watch?v=D5uuyAoRrnw"\n2. "https://www.youtube.com/watch?v=PMh6jkenyLs"\n\n**Summary:**\n\nThe passage provides a brief overview of Unity Catalog\'s capabilities for managing machine learning models and recommends additional resources for those who want to learn more.'

Trace(info=TraceInfo(request_id='tr-0c0eb41deaca4112922a0903921499c3', experiment_id='295899001271457', timestamp_ms=1727761752865, execution_time_ms=9584, status=<TraceStatus.OK: 'OK'>, request_metadata={'mlflow.trace_schema.version': '2', 'mlflow.traceInputs': '"How machine learning models are stored in Unity Catalog?"', 'mlflow.traceOutputs': '"The provided text is a passage that describes how machine learning models are stored and managed in Unity Catalog. Here\'s a breakdown of the content:\\n\\n**Main Points:**\\n\\n1. Machine learning models are stored in Unity Catalog as managed tables,...'}, tags={'mlflow.databricks.notebook.commandID': '7070825677414328105_6273645261473635893_46b8cb5b9e8f4cdb8ed1d8376be2b557', 'mlflow.databricks.notebookID': '295899001271457', 'mlflow.databricks.notebookPath': '/Users/labuser7793232@vocareum.com/generative-ai-application-development-1.1.1/02 - Building Multi-stage Reasoning/2.1 - Building Multi-stage AI Systems', 'mlflow.databricks.webappURL'


## Clean up Classroom

Run the following cell to remove lessons-specific assets created during this lesson.

In [0]:
DA.cleanup()


## Conclusion

In this demo, we explored building a multi-stage reasoning system with Databricks' tools and LangChain. We began by introducing common system components and then focused on creating chains for specific tasks like answering user queries and finding DAIS-2023 talks. By the end, participants learned to use LangChain beyond just LLMs and construct sequential chains for multi-stage analyses.


&copy; 2024 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>